# iTorch / Torch7 implementation


**Gm = d** in lua/torch for LuaJIT. 


## Prerequisites

You need to install `torch` and (if you want to run this notebook) `iTorch`. Works for me on Ubuntu 16.04 and MacOS 10.11. Not gonna say the Mac installation is easy though. Everything was very smooth on Ubuntu.

- [Getting started with Torch](http://torch.ch/docs/getting-started.html)
- [iTorch Requirements](https://github.com/facebook/iTorch#requirements)
- [Torch cheatsheet](https://github.com/torch/torch7/wiki/Cheatsheet)

In [1]:
Plot = require 'itorch.Plot'

In [2]:
-- Implements MATLAB's convmtx.
function convmtx (h, n)
    local m = h:size()[1] - 1
    local c = torch.Tensor(n, n+m):fill(0)
    for i = 1, n do
        c[{ i, {i, i+m} }] = h
    end
    return c
end

In [3]:
-- Implements bruges's ricker.
function ricker (f, length, dt)
    -- Defaults.
    f = f or 25
    length = length or 0.128
    dt = dt or 0.001

    -- Time basis.
    local t = torch.range(0, length, dt) - length/2
    
    -- Amplitudes.
    local x = 1 - 2 * math.pi * f^2 * torch.pow(t, 2)
    local y = torch.exp(-math.pi^2 * f^2 * torch.pow(t, 2))
    local a = torch.cmul(x, y)

    return t, a
end

## Construct the model

In [4]:
--                         VP    RHO
imp = torch.ones(51, 1) * 2550 * 2650
imp[{{11, 15}, {}}] =     2700 * 2750   
imp[{{16, 27}, {}}] =     2400 * 2450
imp[{{28, 35}, {}}] =     2800 * 3000

In [5]:
x = torch.range(1, imp:size(1))
plot = Plot():line(x, imp[{{},1}])
             :title('Impedance')
             :draw()

In [6]:
numerator = imp[{{2, imp:size(1)}}] - imp[{{1, imp:size(1) - 1}}]
denominator = imp[{{2, imp:size(1)}}] + imp[{{1, imp:size(1) - 1}}]
m = torch.cdiv(numerator, denominator)

In [7]:
x = torch.range(1, m:size(1))
plot = Plot():line(x, m[{{}, 1}], 'blue')
             :title('Reflectivity')
             :draw()

## Forward operator: convolution with wavelet

Now we make the kernel matrix *G*, which represents convolution.

In [8]:
t, wavelet = ricker(100, 0.05, 0.001)

In [9]:
plot = Plot():line(t, wavelet)
             :title('Wavelet')
             :draw()

In [10]:
G = convmtx(wavelet, m:size(1))[{{}, {26,75}}]

In [11]:
d = G * m

In [12]:
x = torch.range(1, d:size(1))
plot = Plot():circle(x, d[{{}, 1}])
             :title('Data d')
             :draw()

## Write an array for NumPy to read

In [13]:
torch.save('G_from_torch.txt', G, 'ascii')

## Noise free: minimum norm

I don't understand what's going wrong here. The math is exactly how it is in the NumPy notebook, where this works fine. I think there might be something funny about `torch.inverse`... although it seems to behave in the damped LS example.

So you're better off looking at the Cholesky and QR factorization examples, which ought to give exactly the same result as this.

In [14]:
-- It looks like torch.inverse() might be returning the transpose...
m_est = G:t() * torch.inverse(G * G:t()):t() * d
d_pred = G * m_est

In [15]:
x = torch.range(1, m_est:size(1))
plot = Plot():line(x, m_est[{{}, 1}])
             :title('Predicted model m_est')
             :draw()

In [16]:
x = torch.range(1, d_pred:size(1))
plot = Plot():circle(x, d_pred[{{}, 1}])
             :title('Predicted data d_pred')
             :draw()

## Least squares solution using GESV

This uses LAPACK. It proceeds via factorization instead of the inverse of (G * G.T). It's the equivalent of calling NumPy or SciPy's `linalg.solve` function.

In [21]:
-- Use torch.gels()
m_est = torch.gels(d, G)
d_pred = G * m_est

-- Plot
x = torch.range(1,d_pred:size(1))
plot = Plot():circle(x, d[{{},1}], 'blue')
             :circle(x, d_pred[{{},1}], 'red')
             :title('blue:d   red:d_pred')
             :draw()

In [18]:
d:dist(d_pred)

1.9173170804692e-16	


This is the L2 norm; Mauricio used the square of this as the misfit: 

In [19]:
(d_pred - d):t() * (d_pred - d)

1e-32 *
  3.6761
[torch.DoubleTensor of size 1x1]



## QR factorization

Again, avoiding finding inverses. Uses LAPACK.

In [19]:
q, r = torch.qr(G)
p = q:t() * d
m_est = torch.inverse(r) * p
d_pred = G * m_est

-- Plot
x = torch.range(1,d_pred:size(1))
plot = Plot():circle(x, d[{{},1}], 'blue')
             :circle(x, d_pred[{{},1}], 'red')
             :title('blue:d   red:d_pred')
             :draw()

In [20]:
d:dist(d_pred)

1.8638831565168e-10	


## With noise: damped least squares

In [21]:
-- Add noise
dc = G * m
s = 0.025
d = dc + s * (torch.DoubleTensor(d:size()):uniform(0,1) - 0.5)

In [22]:
-- Compute using Mauricio's second form
I = torch.eye(G:size(1))
mu = 2.5
m_est = G:t() * torch.inverse(G*G:t() + mu*I) * d
d_pred = G * m_est

In [23]:
-- Plot
x = torch.range(1,d_pred:size(1))
plot = Plot():line(x, d[{{},1}], 'blue'):circle(x, d[{{},1}], 'blue')
             :line(x, d_pred[{{},1}]):circle(x, d_pred[{{},1}])
             :title('blue:d   red:d_pred')
             :draw()

## Try using Cholesky decomposition

It's [not a good idea](http://www.johndcook.com/blog/2010/01/19/dont-invert-that-matrix/) to use the matrix inverse. So let's try factorization:

In [24]:
chol = torch.potrf(G * G:t() + mu * I)

In [25]:
m_est = torch.potrs(G:t() * d, chol)

In [26]:
d_pred = G * m_est

In [27]:
-- Plot
x = torch.range(1,d_pred:size(1))
plot = Plot():line(x, d[{{},1}], 'blue'):circle(x, d[{{},1}], 'blue')
             :line(x, d_pred[{{},1}]):circle(x, d_pred[{{},1}])
             :title('blue:d   red:d_pred')
             :draw()

## With noise: damped LS with 1st derivative regularization

In [28]:
h = torch.Tensor({1,-1})
cols = G:size(1) + h:size()[1] - 1
W = convmtx(h, G:size(1))[{ {},{1, cols-1} }]

Now we solve:

$$ \hat{\mathbf{m}} = (\mathbf{G}^\mathrm{T} \mathbf{G} + \mu \mathbf{W}^\mathrm{T} \mathbf{W})^{-1} \mathbf{G}^\mathrm{T} \mathbf{d} \ \ $$

In [29]:
m_est = torch.inverse(G:t()*G + mu*W:t()*W) * G:t() * d
d_pred = G * m_est

In [30]:
-- Plot
x = torch.range(1,d_pred:size(1))
plot = Plot():line(x, d[{{},1}], 'blue'):circle(x, d[{{},1}], 'blue')
             :line(x, d_pred[{{},1}]):circle(x, d_pred[{{},1}])
             :title('blue:d   red:d_pred')
             :draw()

In [31]:
-- Plot
x = torch.range(1,m_est:size(1))
plot = Plot():line(x, m[{{},1}], 'blue', 'blue')
             :line(x, m_est[{{},1}]):title('blue:m   red:m_est')
             :draw()

## Damped LS with 2nd derivative regularization

In [32]:
h = torch.Tensor({1,-2, 1})
W = convmtx(h, G:size(1))[{{}, {2, G:size(1)+1}}]

Q = torch.inverse(W * W:t())
m_est = Q * G:t() * torch.inverse(G * Q * G:t()) * d
d_pred = G * m_est

-- Plot
x = torch.range(1, m_est:size(1))
plot = Plot():line(x, m[{{}, 1}], 'blue', 'blue')
             :line(x, m_est[{{}, 1}]):title('blue:m   red:m_est')
             :draw()